In [1]:
import os
os.environ["CUDA_VISIBLE_DEVICES"]="1"
import pathlib
import numpy as np
import pandas as pd
import sklearn.metrics as metrics
from tensorflow.keras.models import load_model
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.preprocessing import image

/usr/local/lib/python3.6/dist-packages/tensorflow/python/framework/dtypes.py:516: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/usr/local/lib/python3.6/dist-packages/tensorflow/python/framework/dtypes.py:517: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/usr/local/lib/python3.6/dist-packages/tensorflow/python/framework/dtypes.py:518: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
/usr/local/lib/python3.6/dist-packages/tensorflow/python/framework/dtypes.py:519: FutureWarning: Passing (type, 1) or 

In [2]:
data_root_orig = './testData'
data_root = pathlib.Path(data_root_orig)
print(data_root)
label_names = sorted(item.name for item in data_root.glob('*/') if item.is_dir())
print(label_names)
label_to_index = dict((name, i) for i, name in enumerate(label_names))

testData
['GabyNg', 'LeeSeng', 'XiaoYan']


In [3]:
test_set_df = pd.read_csv('./test_GabyNg_face_set.csv')
print(len(test_set_df))

21


In [4]:
pixel = 128
batch_size = 128
tdatagen_rescale = ImageDataGenerator(
    rescale=1./255,
    width_shift_range=0,
    height_shift_range=0,
    rotation_range=0,
    zoom_range=0,
    # shear_range=0.15,
    horizontal_flip=False,
    vertical_flip=False,
    fill_mode='nearest')

In [5]:
filename_rescale = 'model/face_cnn_rescale_model.h5'
face_rescale = load_model(filename_rescale)

Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor
Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor
Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor


In [6]:
test_generator = tdatagen_rescale.flow_from_dataframe(dataframe=test_set_df, x_col="filename",
                                                                y_col="label",
                                                                class_mode="categorical",
                                                                target_size=(pixel, pixel),
                                                                shuffle=False,
                                                                batch_size=128)

Found 21 validated image filenames belonging to 3 classes.


In [7]:
pred_rescale = face_rescale.predict(test_generator)

In [8]:
predout_rescale = np.argmax(pred_rescale, axis=1)
testout_rescale = test_set_df['label'].apply(lambda x: label_to_index[x]).values
print('testout:', testout_rescale)
print('predout:', predout_rescale)

testScores_rescale = metrics.accuracy_score(testout_rescale, predout_rescale)
confusion_rescale = metrics.confusion_matrix(testout_rescale, predout_rescale)

print("Best accuracy (on testing dataset): %.2f%%" % (testScores_rescale * 100))
print(metrics.classification_report(testout_rescale, predout_rescale, labels=[0,1,2],target_names=label_names, digits=4))
print(confusion_rescale)

testout: [0 0 0 0 0 2 2 2 2 2 2 2 2 2 1 1 1 1 1 1 1]
predout: [1 0 0 0 0 2 2 2 2 2 2 2 2 0 1 0 1 1 1 2 1]
Best accuracy (on testing dataset): 80.95%
              precision    recall  f1-score   support

      GabyNg     0.6667    0.8000    0.7273         5
     LeeSeng     0.8333    0.7143    0.7692         7
     XiaoYan     0.8889    0.8889    0.8889         9

    accuracy                         0.8095        21
   macro avg     0.7963    0.8011    0.7951        21
weighted avg     0.8175    0.8095    0.8105        21

[[4 1 0]
 [1 5 1]
 [1 0 8]]


In [9]:
filename_sample = 'model/face_cnn_sample_b128_e100.h5'
face_sample = load_model(filename_sample)

In [10]:
tdatagen_sample = ImageDataGenerator(
    samplewise_center=True,
    samplewise_std_normalization=True,
    width_shift_range=0,
    height_shift_range=0,
    rotation_range=0,
    zoom_range=0,
    shear_range=0,
    horizontal_flip=False,
    vertical_flip=False,
    fill_mode='nearest')

In [11]:
test_generator_sample = tdatagen_sample.flow_from_dataframe(dataframe=test_set_df, x_col="filename",
                                                                y_col="label",
                                                                class_mode="categorical",
                                                                target_size=(pixel, pixel),
                                                                shuffle=False,
                                                                batch_size=128)

Found 21 validated image filenames belonging to 3 classes.


In [12]:
pred_sample = face_sample.predict(test_generator_sample)

In [13]:
predout_sample = np.argmax(pred_sample, axis=1)
testout_sample = test_set_df['label'].apply(lambda x: label_to_index[x]).values
print('testout:', testout_sample)
print('predout:', predout_sample)

testScores_sample = metrics.accuracy_score(testout_sample, predout_sample)
confusion_sample = metrics.confusion_matrix(testout_sample, predout_sample)

print("Best accuracy (on testing dataset): %.2f%%" % (testScores_sample * 100))
print(metrics.classification_report(testout_sample, predout_sample, labels=[0,1,2],target_names=label_names, digits=4))
print(confusion_sample)

testout: [0 0 0 0 0 2 2 2 2 2 2 2 2 2 1 1 1 1 1 1 1]
predout: [1 0 0 0 0 2 2 0 2 2 2 2 2 2 1 1 1 1 1 2 1]
Best accuracy (on testing dataset): 85.71%
              precision    recall  f1-score   support

      GabyNg     0.8000    0.8000    0.8000         5
     LeeSeng     0.8571    0.8571    0.8571         7
     XiaoYan     0.8889    0.8889    0.8889         9

    accuracy                         0.8571        21
   macro avg     0.8487    0.8487    0.8487        21
weighted avg     0.8571    0.8571    0.8571        21

[[4 1 0]
 [0 6 1]
 [1 0 8]]
